In [1]:
import json
import glow
glow.register(spark)

In [7]:
df = spark.read.format("text").load('file:///mnt/SAIGE/extdata/output/example_binary.varianceRatio.txt.sparseGRM.mtx.sample')

In [9]:
df.count()

1000

In [37]:
saige_script = r"""#!/bin/sh
set -e

export tmpdir=$(mktemp -d -t vcf.XXXXXX)
cat - > ${tmpdir}/input.vcf
bgzip -c ${tmpdir}/input.vcf > ${tmpdir}/input.vcf.gz
tabix -p vcf ${tmpdir}/input.vcf.gz

export dataDir='/mnt/SAIGE/extdata'
 
/opt/conda/default/bin/Rscript /mnt/SAIGE/extdata/step2_SPAtests.R \
    --vcfFile=${tmpdir}/input.vcf.gz \
    --vcfFileIndex=${tmpdir}/input.vcf.gz.tbi \
    --vcfField=DS \
    --chrom=1 \
    --minMAF=0.0001 \
    --minMAC=1 \
    --GMMATmodelFile=${dataDir}/output/example_binary.rda \
    --varianceRatioFile=${dataDir}/output/example_binary.varianceRatio.txt \
    --SAIGEOutputFile=${tmpdir}/output.txt \
    --numLinesOutput=2 \
    --IsOutputAFinCaseCtrl=TRUE \
    --IsOutputNinCaseCtrl=TRUE \
    --IsOutputHetHomCountsinCaseCtrl=TRUE \
    >&2
    
cat ${tmpdir}/output.txt
rm -rf ${tmpdir}
"""

In [38]:
cmd = json.dumps(["bash", "-c", saige_script])
#output_df = glow.transform("pipe", input_df, cmd=cmd, input_formatter='vcf', in_vcf_header=input_vcf,
#                           output_formatter='csv', out_header='true', out_delimiter=' ')

In [39]:
print(cmd)

["bash", "-c", "#!/bin/sh\nset -e\n\nexport tmpdir=$(mktemp -d -t vcf.XXXXXX)\ncat - > ${tmpdir}/input.vcf\nbgzip -c ${tmpdir}/input.vcf > ${tmpdir}/input.vcf.gz\ntabix -p vcf ${tmpdir}/input.vcf.gz\n\nexport dataDir='/mnt/SAIGE/extdata'\n \n/opt/conda/default/bin/Rscript /mnt/SAIGE/extdata/step2_SPAtests.R \\\n    --vcfFile=${tmpdir}/input.vcf.gz \\\n    --vcfFileIndex=${tmpdir}/input.vcf.gz.tbi \\\n    --vcfField=DS \\\n    --chrom=1 \\\n    --minMAF=0.0001 \\\n    --minMAC=1 \\\n    --GMMATmodelFile=${dataDir}/output/example_binary.rda \\\n    --varianceRatioFile=${dataDir}/output/example_binary.varianceRatio.txt \\\n    --SAIGEOutputFile=${tmpdir}/output.txt \\\n    --numLinesOutput=2 \\\n    --IsOutputAFinCaseCtrl=TRUE \\\n    --IsOutputNinCaseCtrl=TRUE \\\n    --IsOutputHetHomCountsinCaseCtrl=TRUE \\\n    >&2\n    \ncat ${tmpdir}/output.txt\nrm -rf ${tmpdir}\n"]


In [13]:
input_vcf = "file:///mnt/SAIGE/extdata/input/dosage_10markers.vcf.gz"
input_df = spark.read.format("vcf").load(input_vcf)

In [40]:
output_df = glow.transform("pipe", input_df, cmd=cmd, input_formatter='vcf', in_vcf_header=input_vcf,
                          output_formatter='csv', out_header='true', out_delimiter=' ')

In [41]:
output_df.show()

+---+--------+-----+-------+-------+----------+-------------------+--------------+----+--------------------+-----------------+-------------------+-----------------+-----------------+---------------+------------------+------------------+-------------------+-------------------+-------+----------+------------------+------------------+------------------+------------------+
|CHR|     POS|SNPID|Allele1|Allele2|AC_Allele2|         AF_Allele2|imputationInfo|   N|                BETA|               SE|              Tstat|          p.value|       p.value.NA|Is.SPA.converge|              varT|          varTstar|           AF.Cases|        AF.Controls|N.Cases|N.Controls|homN_Allele2_cases|hetN_Allele2_cases|homN_Allele2_ctrls|hetN_Allele2_ctrls|
+---+--------+-----+-------+-------+----------+-------------------+--------------+----+--------------------+-----------------+-------------------+-----------------+-----------------+---------------+------------------+------------------+-------------------+